In [0]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [0]:
df = pd.read_csv('manipal_sah_tr.csv')
df_test = pd.read_csv('manipal_sah_test.csv')
df.drop(['Radiological Vasospas1', 'Clinical ischemic de0icits', 'Outcome at Discharge'], inplace = True,  axis = 1) 
df_test.drop(['Radiological Vasospas1', 'Clinical ischemic de0icits', 'Outcome at Discharge'], inplace = True,  axis = 1) 

print(df.head())

   Age  Sex  Hypertension  Daibetes  WFNS grade  Outcome at 3 months
0   56    1             0         1           1                    1
1   54    1             0         1           1                    5
2   66    1             0         1           2                    5
3   46    1             0         0           1                    5
4   67    0             1         1           2                    5


In [0]:
X = df.loc[:, 'Age':'WFNS grade']
Y = df.loc[:,'Outcome at 3 months']
Y_GOS = Y-1
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 5 columns):
Age             627 non-null int64
Sex             627 non-null int64
Hypertension    627 non-null int64
Daibetes        627 non-null int64
WFNS grade      627 non-null int64
dtypes: int64(5)
memory usage: 24.6 KB


In [0]:

# # encode class values as integers
# encoder = LabelEncoder()
# encoder.fit(Y)
# encoded_Y = encoder.transform(Y)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)

# print(dummy_y)

In [0]:
# def baseline_model():
# 	# create model
# 	model = Sequential()
# 	model.add(Dense(8, input_dim=5, activation='relu'))
# 	model.add(Dense(5, activation='softmax'))
# 	# Compile model
# 	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#   #model.summary()
# 	return model
 
# estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=5, verbose=0)
# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(estimator, X, Y, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y_GOS,test_size=0.2, random_state=42, stratify=Y)

In [0]:
num_classes=5
model = Sequential()
model.add(Dense(128, activation='relu', input_dim = 5))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               768       
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 645       
Total params: 17,925
Trainable params: 17,925
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
class_weight = {0: 6.,
                1: 6.,
                2: 3.,
                3: 3.,
                4: 1.}

In [0]:
batch_size = 32
num_classes = 5
epochs = 100
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_data=(X_test, y_test),class_weight=class_weight)

Train on 501 samples, validate on 126 samples
Epoch 1/100
 - 0s - loss: 4.0990 - acc: 0.1816 - val_loss: 1.7086 - val_acc: 0.0794
Epoch 2/100
 - 0s - loss: 4.1505 - acc: 0.2176 - val_loss: 1.5943 - val_acc: 0.1984
Epoch 3/100
 - 0s - loss: 4.2333 - acc: 0.2176 - val_loss: 1.5782 - val_acc: 0.1746
Epoch 4/100
 - 0s - loss: 4.2360 - acc: 0.2176 - val_loss: 1.5818 - val_acc: 0.1825
Epoch 5/100
 - 0s - loss: 4.2056 - acc: 0.2096 - val_loss: 1.7514 - val_acc: 0.1429
Epoch 6/100
 - 0s - loss: 4.1645 - acc: 0.2156 - val_loss: 1.5679 - val_acc: 0.2302
Epoch 7/100
 - 0s - loss: 4.1603 - acc: 0.1796 - val_loss: 1.8310 - val_acc: 0.0794
Epoch 8/100
 - 0s - loss: 4.1012 - acc: 0.2156 - val_loss: 1.6819 - val_acc: 0.0873
Epoch 9/100
 - 0s - loss: 4.1098 - acc: 0.2116 - val_loss: 1.4634 - val_acc: 0.3492
Epoch 10/100
 - 0s - loss: 4.0098 - acc: 0.2335 - val_loss: 1.4846 - val_acc: 0.3492
Epoch 11/100
 - 0s - loss: 4.1116 - acc: 0.2834 - val_loss: 1.6479 - val_acc: 0.2143
Epoch 12/100
 - 0s - loss: 4

In [0]:
import numpy as np

In [0]:
np.argmax(model.predict_proba(df_test),axis = 1)


array([4, 4, 4, 1, 4, 4, 4, 1, 0, 0, 0, 4, 4, 4, 0, 0, 4, 0, 4, 4, 4, 4,
       1, 4, 4, 0, 0, 4, 4, 4, 4, 0, 4, 0, 4, 4, 4, 4, 1, 4, 4, 0, 4, 1,
       4, 1, 0, 4, 4, 4, 1, 4, 4, 4, 0, 4, 0, 4, 0, 4, 4, 0, 1, 1, 4, 1,
       4, 4, 4, 0, 4, 4, 4, 0, 4, 1, 4, 0, 4, 4, 1, 4, 4, 4, 1, 4, 4, 4,
       3, 4, 4, 4, 0, 4, 0, 4, 4, 4, 4, 0, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4,
       0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 4, 0, 0, 0, 4, 4, 4, 4, 1, 4,
       4, 0, 4, 4, 4, 0, 0, 1, 4, 4, 0, 4, 4, 4, 1, 1, 4, 1])